In [3]:
%load_ext dotenv

In [4]:
%dotenv

In [5]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [6]:
vectorstore = Chroma(persist_directory= './intro-to-ds-lectures',
                    embedding_function= OpenAIEmbeddings(model='text-embedding-ada-002'))

/var/folders/jw/c29_qvvs3yb90jqlkdbpt4_m0000gn/T/ipykernel_8932/2139955777.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory= './intro-to-ds-lectures',


In [7]:
len(vectorstore.get()['documents'])

21

In [8]:
retriever = vectorstore.as_retriever(search_type = 'mmr',
                                    search_kwargs = {'k':3, 'lambda_mult': 0.7})

In [9]:
TEMPLATE = ''' 
Answer the following question: 
{question}


To answer the question, use only the following context:
{context}


At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures. 
'''


prompt_template = PromptTemplate.from_template(TEMPLATE)

In [10]:
chat = ChatOpenAI(model_name = 'gpt-4',
                 model_kwargs={'seed': 365},
                 max_tokens = 250)

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [11]:
question = "What software do data scientists use?"

In [19]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | chat 
         | StrOutputParser())

In [18]:
chain.invoke(question)

'Data scientists use several software and programming languages such as R, Python, Hadoop, Power BI, SaS, Qlik, and Tableau. The R and Python programming languages are popular because they can manipulate data and are integrated within multiple data and data science software platforms. Hadoop is a software framework designed to handle the complexity and computational intensity of big data by distributing the computational tasks on multiple computers. Power BI, SaS, Qlik, and Tableau are software designed for business intelligence visualizations.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'

In [16]:
print("Data scientists use several software and programming languages such as R, Python, Hadoop, Power BI, SaS, Qlik, and Tableau. The most popular among these are R and Python due to their ability to manipulate data and their integration within multiple data and data science software platforms. These tools are suitable for a wide variety of business and data-related problems from beginning to end. Hadoop is a software framework designed to handle big data and its computational intensity. Power BI, SaS, Qlik, and especially Tableau are notable software designed for business intelligence visualizations. \n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'")



Data scientists use several software and programming languages such as R, Python, Hadoop, Power BI, SaS, Qlik, and Tableau. The most popular among these are R and Python due to their ability to manipulate data and their integration within multiple data and data science software platforms. These tools are suitable for a wide variety of business and data-related problems from beginning to end. Hadoop is a software framework designed to handle big data and its computational intensity. Power BI, SaS, Qlik, and especially Tableau are notable software designed for business intelligence visualizations. 

Resources: Programming Languages & Software Employed in Data Science - All the Tools You Need'
